Created By: Anshoo Mehra
Dated: September 21st 2017

# Tensorflow Guide Chronological Order ..

All Credit to Udacity, all labs & explanation below is excerpt from class labs. Intent is to try all key examples & keep refrence to key highlights for future reference.

I created this notebook for personal reference, however if this helps anyone in general please feel free to refer & consider Udacity policy/lceinses prior sharing ahead. I apologize documentation is fairly limited, it is just to the point to revise concepts than using this as primary guide to learn .. 

### Hello World!

In [1]:
import tensorflow as tf

# Create TensorFlow object called hello_constant
hello_constant = tf.constant('Hello World!')

with tf.Session() as sess:
    # Run the tf.constant operation in the session
    output = sess.run(hello_constant)
    print(output)

b'Hello World!'


### Placeholders (Can't be modified like as Constant shown above)

In [2]:
import tensorflow as tf

def run():
    output = None
    x = tf.placeholder(tf.int32)

    with tf.Session() as sess:
        # TODO: Feed the x tensor 123
        output = sess.run(x, feed_dict={x:123})

    return output

print(run())

123


### Variable

In [3]:
vx = tf.Variable(5)

# The tf.Variable class creates a tensor with an initial value that
# can be modified, much like a normal Python variable. This tensor
# stores its state in the session, so you must initialize the state
# of the tensor manually. You'll use the tf.global_variables_initializer()
# function to initialize the state of all the Variable tensors.

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(vx))
    print(sess.run(vx.assign(123)))
    print(sess.run(vx.assign_add(1)))

5
123
124


### Math Functions

In [4]:
x = tf.add(5, 2) #7
y = tf.subtract(10, 4) # 6
z = tf.multiply(2, 5)  # 10
f = tf.div(4,2)

with tf.Session() as sess:
        x_ = sess.run(x)
        y_ = sess.run(y)
        z_ = sess.run(z)
        f_ = sess.run(f)

print (x_, y_, z_, f_)

7 6 10 2


### Converting Types

In [78]:
#with tf.Session() as sess:
    #sess.run(tf.subtract(tf.constant(2.0),tf.constant(1))) 

## TRY TO UNCOMMENT ABOVE TO SEE THE ERROR MESSAGE..

# Fails with ValueError: Tensor conversion requested dtype float32 for Tensor with dtype int32:
# That's because the constant 1 is an integer but the constant 2.0 
# is a floating point value and subtract expects them to match.# In cases like these, you can either make sure your data is all 
# of the same type, or you can cast a value to another type. In this case,
# converting the 2.0 to an integer before subtracting, like so, will give 
# the correct result:

In [6]:
# In cases like these, you can either make sure your data is all 
# of the same type, or you can cast a value to another type. In this case,
# converting the 2.0 to an integer before subtracting, like so, will give 
# the correct result:

dummy = tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))   # 1

with tf.Session() as sess:
    print (sess.run(dummy))

1


### Example Math Function & Type Conversion

In [7]:
import tensorflow as tf

# TODO: Convert the following to TensorFlow:
#x = 10
#y = 2
#z = x/y - 1

# TODO: Print z from a session
x = tf.constant(10)
y = tf.constant(2)
z = tf.subtract(tf.div(x,y),tf.constant(1))
f = tf.cast(z, tf.float32)
## OR z = tf.div(x,y) - 1

with tf.Session() as sess:
    output = sess.run(z)
    print(output)
    output = sess.run(f)
    print(output)
    

4
4.0


### Assuming everyone knows Linear Combination Equation y = xW + b 

y = Logits
x = Inputs
W = Weights
b = Bias

Linear Equation will give is Logits

### Python Softmax

Softmax Function Convert Logits to Probabilities (All Probabilities Add to 1) 

In [18]:
import numpy as np


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    # TODO: Compute and return softmax(x)
    return np.exp(x)/np.sum(np.exp(x), axis=0)
        
logits = [3.0, 1.0, 0.2]
print(softmax(logits))

[ 0.8360188   0.11314284  0.05083836]


### Softmax in Tensorflow

In [21]:
import tensorflow as tf


def run():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)

    softmax = tf.nn.softmax(logits)

    with tf.Session() as sess:
        output = sess.run(softmax, feed_dict={logits: logit_data})

    return output

print(run())

[ 0.65900117  0.24243298  0.09856589]


### One-Hot Encoding 

y = [ .5, .6, .9]

y_one_hot_encoded = [ 0, 0, 1]

So the hightest probability gets marked 1 and rest 0 as simple as that ...

One Hot Encoding works really well but One problem with One Hot Encoding is that once classes grow to thousands or millions, vector becomes really large & very inefficient. This can be solved with embeddings, we will see that later.

However, this method makes the process very simple, we have just 2 vectors, one vector contains probabilities of classes and other vector one-hot encodeding for labels. So if we have a way to measure distance between these two Vectors, that can help us classify and this process of measuring distance is called **Cross-Entropy.

**Cross-Entropy will give us small distance for Right Class and higher distance for Incorrect Classes.**

### Cross Entropy

D(S,L) = - Σᵢ Lᵢ * log(Sᵢ)



### Overall Process is called " Multinomial Logistics Classification "


INPUT **--linear model--**> LOGITS > **--softmax--** > PROBABILITIES > **--cross_entropy--** < 1_HOT_LABELS





#### Let's see how to compute Cross Entropy in Tesnforflow 

In [22]:
import tensorflow as tf

softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

# ToDo: Print cross entropy from session
cross_entropy = -tf.reduce_sum(tf.multiply(one_hot, tf.log(softmax)))

with tf.Session() as sess:
    print(sess.run(cross_entropy, feed_dict={softmax: softmax_data, one_hot: one_hot_data}))


0.356675


### Almost there, since most of us will be trying to train models on CPU, concept called mini-batching is something we must learn to not run short on memory and be stranded in middle of nowhere ..

Mini-batching is a technique for training on subsets of the dataset instead of all the data at one time. This provides the ability to train a model, even if a computer lacks the memory to store the entire dataset.

Mini-batching is computationally inefficient, since you can't calculate the loss simultaneously across all samples. However, this is a small price to pay in order to be able to run the model at all.

It's also quite useful combined with SGD. The idea is to randomly shuffle the data at the start of each epoch, then create the mini-batches. For each mini-batch, you train the network weights with gradient descent. Since these batches are random, you're performing SGD with each batch.

**TensorFlow Mini-batching**
In order to use mini-batching, you must first divide your data into batches.

Unfortunately, it's sometimes impossible to divide the data into batches of exactly equal size. For example, imagine you'd like to create batches of 128 samples each from a dataset of 1000 samples. Since 128 does not evenly divide into 1000, you'd wind up with 7 batches of 128 samples, and 1 batch of 104 samples. (7*128 + 1*104 = 1000)

In that case, the size of the batches would vary, so you need to take advantage of TensorFlow's tf.placeholder() function to receive the varying batch sizes.

Continuing the example, if each sample had n_input = 784 features and n_classes = 10 possible labels, the dimensions for features would be [None, n_input] and labels would be [None, n_classes].

features = tf.placeholder(tf.float32, [**None**, n_input]) <br>
labels = tf.placeholder(tf.float32, [**None**, n_classes])

What does None do here?

The None dimension is a placeholder for the batch size. At runtime, TensorFlow will accept any batch size greater than 0.

Going back to our earlier example, this setup allows you to feed features and labels into the model as either the batches of 128 samples or the single batch of 104 samples.

Let's understand it better with a example .. 

In [53]:
import math
from pprint import pprint


def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches


# 4 Samples of features
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]

# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

example_batches = batches(3, example_features, example_labels)

pprint(example_batches)

[[[['F11', 'F12', 'F13', 'F14'],
   ['F21', 'F22', 'F23', 'F24'],
   ['F31', 'F32', 'F33', 'F34']],
  [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]],
 [[['F41', 'F42', 'F43', 'F44']], [['L41', 'L42']]]]


### Epochs 

With mini-batching the accuracy is low, but you probably know that you could train on the dataset more than once. You can train a model using the dataset multiple times.

An **epoch** is a single forward and backward pass of the whole dataset. This is used to increase the accuracy of the model without requiring more data. This section will cover epochs in TensorFlow and how to choose the right number of epochs.

## So, let's put it all together and run the below lab. 

**If it appear confusing, try to going to last commneted lab & understand one step at a time & if need be going back to above examples .. 

### Handwritten Numbers (Image) Classification using MNIST database by applying ..
### Linear Function / Logistics Regression ( y= xW + b )with Tensforflow


Initializing the weights with random numbers from a normal distribution is good practice. Randomizing the weights helps the model from becoming stuck in the same place every time you train it.

Similarly, choosing weights from a normal distribution prevents any one weight from overwhelming other weights. You'll use the tf.truncated_normal() function to generate random numbers from a normal distribution.

The tf.truncated_normal() function returns a tensor with random values from a normal distribution whose magnitude is no more than 2 standard deviations from the mean.

Since the weights are already helping prevent the model from getting stuck, you don't need to randomize the bias.

In [67]:
import tensorflow as tf

def weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    # TODO: Return weights
    return tf.Variable(tf.random_normal([n_features, n_labels]))


def biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    # TODO: Return biases
    return tf.Variable(tf.random_normal[n_labels])


def linear(input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    # TODO: Linear Function (xW + b)
    return tf.add(tf.matmul(input, w), b)

def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))
    
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches

In [75]:
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 128
epochs = 500
learn_rate = 0.09

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Epoch: 0    - Cost: 1.78     Valid Accuracy: 0.727
Epoch: 1    - Cost: 1.27     Valid Accuracy: 0.799
Epoch: 2    - Cost: 1.06     Valid Accuracy: 0.824
Epoch: 3    - Cost: 0.939    Valid Accuracy: 0.841
Epoch: 4    - Cost: 0.85     Valid Accuracy: 0.852
Epoch: 5    - Cost: 0.783    Valid Accuracy: 0.86 
Epoch: 6    - Cost: 0.731    Valid Accuracy: 0.866
Epoch: 7    - Cost: 0.69     Valid Accuracy: 0.87 
Epoch: 8    - Cost: 0.656    Valid Accuracy: 0.874
Epoch: 9    - Cost: 0.629    Valid Accuracy: 0.879
Epoch: 10   - Cost: 0.607    Valid Accuracy: 0.881
Epoch: 11   - Cost: 0.587    Valid Accuracy: 0.883
Epoch: 12   - Cost: 0.57     Valid Accuracy: 0.886
Epoch: 13   - Cost: 0.554    Valid Accuracy: 0.889
Epoc

In [68]:
### SOMETHING TO TRY TO GET HANG OF IT ... IF ABOVE IS TOO COMPLEX .. 

# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# ## Above Block Helper Functions can be modlarized to separate file .. if so
# ## below import
# #from quiz import weights, biases, linear 

# def mnist_features_labels(n_labels):
#     """
#     Gets the first <n> labels from the MNIST dataset
#     :param n_labels: Number of labels to use
#     :return: Tuple of feature list and label list
#     """
#     mnist_features = []
#     mnist_labels = []

#     #mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)
#     mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

#     # In order to make quizzes run faster, we're only looking at 10000 images
#     for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):
        
#         #print ("Before Checking", mnist_feature, mnist_label)
        
#         # Add features and labels if it's for the first <n>th labels
#         # In this eample since n_labels is 3, ie. features & labels for only numbers 0, 1, 2 will be retrieved
#         # from dataset remember each feature is 28*28 image and is flattened to 784 (1*784 Matrix)
#         if mnist_label[:n_labels].any():
#             #If Label is any of top 3, add feature for current index record
#             mnist_features.append(mnist_feature)
#             #If Label is any of top 3, add label values for just top 3 indexes, we do not need rest as we are not
#             # looking to classify anything after number 3..
#             mnist_labels.append(mnist_label[:n_labels])

#     return mnist_features, mnist_labels


# # Number of features (28*28 image is 784 features)
# n_features = 784
# # Number of labels
# n_labels = 3

# # Features and Labels
# features = tf.placeholder(tf.float32)
# labels = tf.placeholder(tf.float32)

# # Weights and Biases
# w = weights(n_features, n_labels)
# b = biases(n_labels)

# # Linear Function xW + b
# logits = linear(features, w, b)


# # Training data
# train_features, train_labels = mnist_features_labels(n_labels)

# with tf.Session() as session:
#     session.run(tf.global_variables_initializer())

#     # Softmax (Thus far we studied Sigmoid, it will be good to google understanding differences between them)
#     prediction = tf.nn.softmax(logits)

#     # Cross entropy
#     # This quantifies how far off the predictions were.
#     # You'll learn more about this in future lessons.
#     cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

#     # Training loss
#     # You'll learn more about this in future lessons.
#     loss = tf.reduce_mean(cross_entropy)

#     # Rate at which the weights are changed
#     # You'll learn more about this in future lessons.
#     learning_rate = 0.08

#     # Gradient Descent
#     # This is the method used to train the model
#     # You'll learn more about this in future lessons.
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

#     # Run optimizer and get loss
#     _, l = session.run(
#         [optimizer, loss],
#         feed_dict={features: train_features, labels: train_labels})

# # Print loss
# print('Loss: {}'.format(l))

## RELU (Rectified Linear Units) 

So far we have build classifier which is very robust for Linear Operations but are limited and can't perform non-linear operations example, additive operation X1 + X2 can be performed but X1 * X2 can't be performed.

Being linear in nature it has many advantages:
- Big Matrix multiplies is exactly what GPUs are designed for, so can run fairly efficiently.
- Linear Operations are very stable, we can mathamatically demonstrate that small changes in input reflect small changes in output & will not spike to large outputs arbitrarily.
- Derivaties are nice as well for Linear Functions, they demostrate constant output & hence super stable.

**So we would like to have our parameters be inside big linear functions but we would also like our Model to perform non-linear operations to solve more complex problems ..**  

One of the ways we can introduce non-linearalities to our model is by injecting RELUs to the our model. RELUs are fairly simple, they are linear when x >0 and 0 everywhere else ..Also, when x < 0 derivative is also, and when x > 0 derivative is constant.

So to implement this in practice without complicating what we have studied..

We will take our INPUTS+WEIGHT+BIAS >>FEED>> RELU LAYERS >>FEED>> CLASSIFIER ..

This simple approach helps us make the whole model non-linear && as added advantage, we will have another parameter to fine tune, i.e. RELU LAYERS (denoted as H).. This is just simple example we can make these layers as deep we want .. 

The above network example applies RELU function to the hidden_layer, effectively turning off any negative weights and acting like an on/off switch. Adding additional layers, like the output layer, after an activation function turns the model into a nonlinear function. This nonlinearity allows the network to solve more complex problems.

**Congrats, with this we will build our first Neural Network using Tensorflow..**

In [79]:
import tensorflow as tf

output = None
hidden_layer_weights = [
    [0.1, 0.2, 0.4],
    [0.4, 0.6, 0.6],
    [0.5, 0.9, 0.1],
    [0.8, 0.2, 0.8]]
out_weights = [
    [0.1, 0.6],
    [0.2, 0.1],
    [0.7, 0.9]]

# Weights and biases
weights = [
    tf.Variable(hidden_layer_weights),
    tf.Variable(out_weights)]
biases = [
    tf.Variable(tf.zeros(3)),
    tf.Variable(tf.zeros(2))]

# Input
features = tf.Variable([[1.0, 2.0, 3.0, 4.0], [-1.0, -2.0, -3.0, -4.0], [11.0, 12.0, 13.0, 14.0]])

# TODO: Create Model
hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
hidden_layer = tf.nn.relu(hidden_layer)
logits = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])

# TODO: Print session results
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(logits))



[[  5.11000013   8.44000053]
 [  0.           0.        ]
 [ 24.01000214  38.23999786]]


### With concepts cleared out, let's re-build classifier we build earlier to Classify Letters over MNIST database using Deep Network with RELUs

In [85]:
from tensorflow.examples.tutorials.mnist import input_data
#You'll use the MNIST dataset provided by TensorFlow, 
# which batches and One-Hot encodes the data for you.

mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.1
training_epochs = 100
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

#The variable n_hidden_layer determines the size of the hidden
#layer in the neural network.This is also known as the width of a layer.
n_hidden_layer = 256 # layer number of features

# Deep neural networks use multiple layers with each layer requiring
# it's own weight and bias. The 'hidden_layer' weight and bias is for
# the hidden layer. The 'out' weight and bias is for the output layer.
# If the neural network were deeper, there would be weights and biases
# for each additional layer.
# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# tf Graph input
x = tf.placeholder("float", [None, 28, 28, 1])
y = tf.placeholder("float", [None, n_classes])

# The MNIST data is made up of 28px by 28px images with a single channel.
# The tf.reshape() function above reshapes the 28px by 28px matrices in x
# into row vectors of 784px.
x_flat = tf.reshape(x, [-1, n_input])

# You've seen the linear function 
# tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
# before, also known as xw + b. Combining linear functions together using
# a ReLU will give you a two layer network.
# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.matmul(layer_1, weights['out']) + biases['out']


# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
#             The MNIST library in TensorFlow provides the ability to
#             receive the dataset in batches. Calling the
#             mnist.train.next_batch() function returns a subset of
#             the training data.
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        # Display logs per epoch step
        if epoch % display_step == 0:
            c = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(c))
    
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # Decrease test_size if you don't have enough memory
    test_size = 256
    print("Accuracy:", accuracy.eval({x: mnist.test.images[:test_size], y: mnist.test.labels[:test_size]}))


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost= 1.249434471
Epoch: 0002 cost= 1.622592926
Epoch: 0003 cost= 0.485644400
Epoch: 0004 cost= 0.066994786
Epoch: 0005 cost= 0.506596088
Epoch: 0006 cost= 0.166660339
Epoch: 0007 cost= 0.256506383
Epoch: 0008 cost= 0.263688326
Epoch: 0009 cost= 0.333746374
Epoch: 0010 cost= 0.026917888
Epoch: 0011 cost= 0.496636808
Epoch: 0012 cost= 0.096257575
Epoch: 0013 cost= 0.130075336
Epoch: 0014 cost= 0.016562125
Epoch: 0015 cost= 0.442557871
Epoch: 0016 cost= 0.000909301
Epoch: 0017 cost= 0.050810650
Epoch: 0018 cost= 0.119163290
Epoch: 0019 cost= 0.095426545
Epoch: 0020 cost= 0.048861191
Epoch: 0021 cost= 0.032808296
Epoch: 0022 cost= 0.000551221
Epoch: 0023 cost= 0.004125504
Epoch: 0024 cost= 0.014760466
Epoch: 0025 cost= 0.011153132
Epoch: 0026 cost= 0.075658090
Epoch: 0027 cost= 0.005784245
Epoch: 0028 cost= 0.007925835
Ep

### Training model could take awfully long time, hence would not wantto loose outr trained model. Thankfully Tensorflow let us save an restore model.. Let's see simple example again with save n restore of model parameters/variables.

In [86]:
# Remove previous Tensors and Operations
tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

learning_rate = 0.09
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('.', one_hot=True)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [88]:
import math

save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)

        # Loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict={features: batch_features, labels: batch_labels})

        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))

    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.10559999942779541
Epoch 10  - Validation Accuracy: 0.24699999392032623
Epoch 20  - Validation Accuracy: 0.4171999990940094
Epoch 30  - Validation Accuracy: 0.5181999802589417
Epoch 40  - Validation Accuracy: 0.5831999778747559
Epoch 50  - Validation Accuracy: 0.6276000142097473
Epoch 60  - Validation Accuracy: 0.6592000126838684
Epoch 70  - Validation Accuracy: 0.6855999827384949
Epoch 80  - Validation Accuracy: 0.704800009727478
Epoch 90  - Validation Accuracy: 0.7203999757766724
Trained Model Saved.


In [89]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: mnist.test.images, labels: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
Test Accuracy: 0.7242000102996826


### Loading the Weights and Biases into a New Model

Sometimes you might want to adjust, or "finetune" a model that you have already trained and saved.

However, loading saved Variables directly into a modified model can generate errors. Let's go over how to avoid these problems.

Naming Error<br>
TensorFlow uses a string identifier for Tensors and Operations called name. If a name is not given, TensorFlow will create one automatically. TensorFlow will give the first node the name <Type>, and then give the name <Type>_<number> for the subsequent nodes. Let's see how this can affect loading a model with a different order of weights and bias:

In [90]:
import tensorflow as tf

# Remove the previous weights and bias
tf.reset_default_graph()

save_file = 'model.ckpt'

# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Save Weights: {}'.format(weights.name))
print('Save Bias: {}'.format(bias.name))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, save_file)

# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
bias = tf.Variable(tf.truncated_normal([3]))
weights = tf.Variable(tf.truncated_normal([2, 3]))

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Load Weights: {}'.format(weights.name))
print('Load Bias: {}'.format(bias.name))

with tf.Session() as sess:
    # Load the weights and bias - ERROR
    saver.restore(sess, save_file)

Save Weights: Variable:0
Save Bias: Variable_1:0
Load Weights: Variable_1:0
Load Bias: Variable:0
INFO:tensorflow:Restoring parameters from model.ckpt


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [3] rhs shape= [2,3]
	 [[Node: save/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable, save/RestoreV2)]]

Caused by op 'save/Assign', defined at:
  File "/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.st

You'll notice that the name properties for weights and bias are different than when you saved the model. This is why the code produces the "Assign requires shapes of both tensors to match" error. The code saver.restore(sess, save_file) is trying to load weight data into bias and bias data into weights.

Instead of letting TensorFlow set the name property, let's set it manually:

In [91]:
import tensorflow as tf

tf.reset_default_graph()

save_file = 'model.ckpt'

# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]), name='weights_0')
bias = tf.Variable(tf.truncated_normal([3]), name='bias_0')

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Save Weights: {}'.format(weights.name))
print('Save Bias: {}'.format(bias.name))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, save_file)

# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
bias = tf.Variable(tf.truncated_normal([3]), name='bias_0')
weights = tf.Variable(tf.truncated_normal([2, 3]) ,name='weights_0')

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Load Weights: {}'.format(weights.name))
print('Load Bias: {}'.format(bias.name))

with tf.Session() as sess:
    # Load the weights and bias - No Error
    saver.restore(sess, save_file)

print('Loaded Weights and Bias successfully.')

Save Weights: weights_0:0
Save Bias: bias_0:0
Load Weights: weights_0:0
Load Bias: bias_0:0
INFO:tensorflow:Restoring parameters from model.ckpt
Loaded Weights and Bias successfully.


### L2 Regularization To Avoid Overfitting

**First Method** - It does not change network model,it is simply adding another term to the overall loss which penalize large weights. Another hyper parameter to tune unfortunately.

**Second Method - Dropouts** - It has become very popular, basically in the network intermediate activation nodes, drop half or certain percentage of nodes to value zero, it force network to have redundant representation as network can't rely on activations all the time and is force to learn from others, in other words it is forced learn to take consensus from all nodes ensuring targets are achieved.

Certainly evaluation can't depend randomness of dropouts we want something deterministic, hence to get consensus opinion, we have to average activations. Make sure to scale remaining activations by factor of 2..

In [92]:
import tensorflow as tf

hidden_layer_weights = [
    [0.1, 0.2, 0.4],
    [0.4, 0.6, 0.6],
    [0.5, 0.9, 0.1],
    [0.8, 0.2, 0.8]]
out_weights = [
    [0.1, 0.6],
    [0.2, 0.1],
    [0.7, 0.9]]

# Weights and biases
weights = [
    tf.Variable(hidden_layer_weights),
    tf.Variable(out_weights)]
biases = [
    tf.Variable(tf.zeros(3)),
    tf.Variable(tf.zeros(2))]

# Input
features = tf.Variable([[0.0, 2.0, 3.0, 4.0], [0.1, 0.2, 0.3, 0.4], [11.0, 12.0, 13.0, 14.0]])

# TODO: Create Model with Dropout
keep_prob = tf.placeholder(tf.float32)
hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
hidden_layer = tf.nn.relu(hidden_layer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)

logits = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])

# TODO: Print logits from a session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(logits, feed_dict={keep_prob: 0.5}))



[[  6.57999945   8.45999908]
 [  0.           0.        ]
 [ 14.28000069  33.09999847]]


# Time to learn about CNNs

Covnets are networks which build on the idea to share parameters across space ..

As simple as example, if we are learning to classify image of cats, it does not matter if the image is on left corner or right corner of image, but if we do not share the weights and ask network to individually identify cats on left and cats on right, it has to do lot of work to achieve this goal which sounds unnecessary in this example..

Similarly, if we are training on text, meaning of Cat does not change at every occurence, hence it will make sense to share weights for classification of cat, however for text based learning we use method called embeddings or RNNs, we will talk more on this subject in later sections.

At the end of CNNs as output, all the spatial information is squeezed out and only content is left. As example : Say it start with with full image & progress as mentioned, if you notice depth is increasing with every step emphasizing content 256x256xRGB >> 128x128x16 >> 64x64x64 >> 32x32x256 >> Classifier

Lingo: 

PATCH == KERNEL == FILTER ==  IS NETWORK LAYER YOU RUN AS SUBSET

FEATURE MAP == EACH SET OF FEATURES, in image with RGB, each R, G, B space is seperate feature map and applying 

PATCH/KERNEL we output/transform too N FEATURE MAPS. So in this example, trasforming 3 Feature Maps to K Feature Maps.

STRIDE: Number of Pixels being shifted each time KERNEL Moves.<br>
Stride of One means: Output is roughly same size as input. <br>
Stride of Two means: Output is roughly half the size of input.<br>
Roughly is used as it depends on how edging is processed, if filter do not pass the image edge it is called 'VALID PADDING', if Filter go beyond image edge, padding is needed to the input image to match the output and is called 'SAME PADDING'

Filters are very agrressive in nature and cause information loss, couple techniques whcih can be applied to minimize loss are ** Max Pooling and Average Pooling**. Idea is to run 1 Stride but somehow combine similar information features/keep relevant features eventually shrinking the size of layer as CNN would do with higher depth. How it performs this is beyond scope here ..

Recently, pooling layers have fallen out of favor. Some reasons are:

- Recent datasets are so big and complex we're more concerned about underfitting.
- Dropout is a much better regularizer.
- Pooling results in a loss of information. Think about the max pooling operation as an example. We only keep the largest of n numbers, thereby disregarding n-1 numbers completely.

**LENET-5 & ALEXNET are good reads on how they best make use of these concepets .. **

**1x1 Covnets** - Helps coverting Linear Model of Convolutions to deeper Non-Linear..

**Inception Module**So we have so many choices as listed above, Inception Module is stack of combination - what it does is compose the mdule in the following way using all above:
- Average Pooling followed by 1x1
- 1x1 Conv
- 1x1 followed by 3x3
- 1x1 followed by 5x5
At the top concatinate the output from all above mentioned as one module .. 

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True, reshape=False)

import tensorflow as tf

#data
#mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Epoch  1, Batch   1 - Loss: 77001.3125 Validation Accuracy: 0.152344
Epoch  1, Batch   2 - Loss: 54442.8828 Validation Accuracy: 0.152344
Epoch  1, Batch   3 - Loss: 39964.0156 Validation Accuracy: 0.140625
Epoch  1, Batch   4 - Loss: 36289.2656 Validation Accuracy: 0.148438
Epoch  1, Batch   5 - Loss: 37250.7773 Validation Accuracy: 0.179688
Epoch  1, Batch   6 - Loss: 29698.1523 Validation Accuracy: 0.199219
Epoch  1, Batch   7 - Loss: 29642.5586 Validation Accuracy: 0.191406
Epoch  1, Batch   8 - Loss: 28352.6934 Validation Accuracy: 0.199219
Epoch  1, Batch   9 - Loss: 24100.8594 Validation Accuracy: 0.210938
Epoch  1, Batch  10 - Loss: 26219.5645 Validation Accuracy: 0.191406
Epoch  1, Batch  11 - Loss: 

Epoch  1, Batch 116 - Loss:  5350.3320 Validation Accuracy: 0.589844
Epoch  1, Batch 117 - Loss:  4975.5225 Validation Accuracy: 0.589844
Epoch  1, Batch 118 - Loss:  4816.0186 Validation Accuracy: 0.613281
Epoch  1, Batch 119 - Loss:  4516.6782 Validation Accuracy: 0.617188
Epoch  1, Batch 120 - Loss:  5973.2788 Validation Accuracy: 0.613281
Epoch  1, Batch 121 - Loss:  5220.0732 Validation Accuracy: 0.609375
Epoch  1, Batch 122 - Loss:  4716.8896 Validation Accuracy: 0.613281
Epoch  1, Batch 123 - Loss:  5006.6011 Validation Accuracy: 0.613281
Epoch  1, Batch 124 - Loss:  6145.7373 Validation Accuracy: 0.609375
Epoch  1, Batch 125 - Loss:  5559.1699 Validation Accuracy: 0.613281
Epoch  1, Batch 126 - Loss:  3920.5278 Validation Accuracy: 0.609375
Epoch  1, Batch 127 - Loss:  5595.6968 Validation Accuracy: 0.621094
Epoch  1, Batch 128 - Loss:  5206.4297 Validation Accuracy: 0.613281
Epoch  1, Batch 129 - Loss:  4634.2490 Validation Accuracy: 0.605469
Epoch  1, Batch 130 - Loss:  5783.

Epoch  1, Batch 235 - Loss:  3676.8247 Validation Accuracy: 0.679688
Epoch  1, Batch 236 - Loss:  3429.7500 Validation Accuracy: 0.683594
Epoch  1, Batch 237 - Loss:  2507.9690 Validation Accuracy: 0.683594
Epoch  1, Batch 238 - Loss:  3406.7322 Validation Accuracy: 0.695312
Epoch  1, Batch 239 - Loss:  3769.9946 Validation Accuracy: 0.699219
Epoch  1, Batch 240 - Loss:  2092.0459 Validation Accuracy: 0.695312
Epoch  1, Batch 241 - Loss:  2531.0740 Validation Accuracy: 0.687500
Epoch  1, Batch 242 - Loss:  2430.4731 Validation Accuracy: 0.695312
Epoch  1, Batch 243 - Loss:  2416.0869 Validation Accuracy: 0.691406
Epoch  1, Batch 244 - Loss:  2989.4473 Validation Accuracy: 0.691406
Epoch  1, Batch 245 - Loss:  2159.5251 Validation Accuracy: 0.683594
Epoch  1, Batch 246 - Loss:  3192.8699 Validation Accuracy: 0.691406
Epoch  1, Batch 247 - Loss:  2476.9446 Validation Accuracy: 0.691406
Epoch  1, Batch 248 - Loss:  2637.2134 Validation Accuracy: 0.691406
Epoch  1, Batch 249 - Loss:  2828.

Epoch  1, Batch 354 - Loss:  1869.6760 Validation Accuracy: 0.710938
Epoch  1, Batch 355 - Loss:  1724.4590 Validation Accuracy: 0.707031
Epoch  1, Batch 356 - Loss:  1969.2212 Validation Accuracy: 0.710938
Epoch  1, Batch 357 - Loss:  2367.9307 Validation Accuracy: 0.714844
Epoch  1, Batch 358 - Loss:  2025.3516 Validation Accuracy: 0.718750
Epoch  1, Batch 359 - Loss:  2235.3105 Validation Accuracy: 0.718750
Epoch  1, Batch 360 - Loss:  2776.2827 Validation Accuracy: 0.718750
Epoch  1, Batch 361 - Loss:  1723.9233 Validation Accuracy: 0.718750
Epoch  1, Batch 362 - Loss:  1492.4822 Validation Accuracy: 0.718750
Epoch  1, Batch 363 - Loss:  1947.5896 Validation Accuracy: 0.718750
Epoch  1, Batch 364 - Loss:  1967.1422 Validation Accuracy: 0.714844
Epoch  1, Batch 365 - Loss:  2563.9111 Validation Accuracy: 0.714844
Epoch  1, Batch 366 - Loss:  1922.9376 Validation Accuracy: 0.714844
Epoch  1, Batch 367 - Loss:  2447.7720 Validation Accuracy: 0.714844
Epoch  1, Batch 368 - Loss:  1940.

Epoch  2, Batch  44 - Loss:  1401.5270 Validation Accuracy: 0.730469
Epoch  2, Batch  45 - Loss:  1405.6310 Validation Accuracy: 0.726562
Epoch  2, Batch  46 - Loss:  1216.3828 Validation Accuracy: 0.722656
Epoch  2, Batch  47 - Loss:  1684.5850 Validation Accuracy: 0.726562
Epoch  2, Batch  48 - Loss:  1306.6262 Validation Accuracy: 0.718750
Epoch  2, Batch  49 - Loss:  1864.8462 Validation Accuracy: 0.730469
Epoch  2, Batch  50 - Loss:  1078.4752 Validation Accuracy: 0.730469
Epoch  2, Batch  51 - Loss:  1487.2583 Validation Accuracy: 0.730469
Epoch  2, Batch  52 - Loss:  2183.1343 Validation Accuracy: 0.730469
Epoch  2, Batch  53 - Loss:  2086.9058 Validation Accuracy: 0.738281
Epoch  2, Batch  54 - Loss:  1696.2123 Validation Accuracy: 0.742188
Epoch  2, Batch  55 - Loss:  1355.5872 Validation Accuracy: 0.738281
Epoch  2, Batch  56 - Loss:  1440.7804 Validation Accuracy: 0.738281
Epoch  2, Batch  57 - Loss:  1378.8058 Validation Accuracy: 0.738281
Epoch  2, Batch  58 - Loss:  1778.

...

Epoch  9, Batch 373 - Loss:   284.7850 Validation Accuracy: 0.828125
Epoch  9, Batch 374 - Loss:   239.9878 Validation Accuracy: 0.832031
Epoch  9, Batch 375 - Loss:   300.4342 Validation Accuracy: 0.828125
Epoch  9, Batch 376 - Loss:   161.1885 Validation Accuracy: 0.828125
Epoch  9, Batch 377 - Loss:   253.6481 Validation Accuracy: 0.832031
Epoch  9, Batch 378 - Loss:   319.4118 Validation Accuracy: 0.832031
Epoch  9, Batch 379 - Loss:   412.0541 Validation Accuracy: 0.835938
Epoch  9, Batch 380 - Loss:   303.7771 Validation Accuracy: 0.828125
Epoch  9, Batch 381 - Loss:   315.6238 Validation Accuracy: 0.828125
Epoch  9, Batch 382 - Loss:   313.6613 Validation Accuracy: 0.820312
Epoch  9, Batch 383 - Loss:   147.2929 Validation Accuracy: 0.828125
Epoch  9, Batch 384 - Loss:   258.0200 Validation Accuracy: 0.832031
Epoch  9, Batch 385 - Loss:   260.1269 Validation Accuracy: 0.835938
Epoch  9, Batch 386 - Loss:   121.5603 Validation Accuracy: 0.835938
Epoch  9, Batch 387 - Loss:   288.

Epoch 10, Batch  63 - Loss:   184.8791 Validation Accuracy: 0.843750
Epoch 10, Batch  64 - Loss:   276.8091 Validation Accuracy: 0.843750
Epoch 10, Batch  65 - Loss:   170.9408 Validation Accuracy: 0.843750
Epoch 10, Batch  66 - Loss:   121.3169 Validation Accuracy: 0.843750
Epoch 10, Batch  67 - Loss:   326.8722 Validation Accuracy: 0.843750
Epoch 10, Batch  68 - Loss:   242.5665 Validation Accuracy: 0.843750
Epoch 10, Batch  69 - Loss:   313.8791 Validation Accuracy: 0.839844
Epoch 10, Batch  70 - Loss:   131.7284 Validation Accuracy: 0.839844
Epoch 10, Batch  71 - Loss:   116.7048 Validation Accuracy: 0.832031
Epoch 10, Batch  72 - Loss:   149.1029 Validation Accuracy: 0.832031
Epoch 10, Batch  73 - Loss:   198.1626 Validation Accuracy: 0.839844
Epoch 10, Batch  74 - Loss:   180.6437 Validation Accuracy: 0.839844
Epoch 10, Batch  75 - Loss:   177.5403 Validation Accuracy: 0.839844
Epoch 10, Batch  76 - Loss:   219.5627 Validation Accuracy: 0.839844
Epoch 10, Batch  77 - Loss:   399.

Epoch 10, Batch 182 - Loss:   199.5548 Validation Accuracy: 0.832031
Epoch 10, Batch 183 - Loss:   214.7185 Validation Accuracy: 0.832031
Epoch 10, Batch 184 - Loss:   196.7523 Validation Accuracy: 0.832031
Epoch 10, Batch 185 - Loss:   320.5714 Validation Accuracy: 0.832031
Epoch 10, Batch 186 - Loss:   355.1827 Validation Accuracy: 0.832031
Epoch 10, Batch 187 - Loss:   111.4176 Validation Accuracy: 0.832031
Epoch 10, Batch 188 - Loss:   181.3265 Validation Accuracy: 0.832031
Epoch 10, Batch 189 - Loss:   247.8883 Validation Accuracy: 0.828125
Epoch 10, Batch 190 - Loss:   192.8175 Validation Accuracy: 0.828125
Epoch 10, Batch 191 - Loss:   158.0123 Validation Accuracy: 0.828125
Epoch 10, Batch 192 - Loss:   152.8733 Validation Accuracy: 0.828125
Epoch 10, Batch 193 - Loss:   333.6830 Validation Accuracy: 0.828125
Epoch 10, Batch 194 - Loss:   283.4893 Validation Accuracy: 0.828125
Epoch 10, Batch 195 - Loss:   250.2013 Validation Accuracy: 0.828125
Epoch 10, Batch 196 - Loss:    82.

Epoch 10, Batch 301 - Loss:   182.7169 Validation Accuracy: 0.847656
Epoch 10, Batch 302 - Loss:   137.4120 Validation Accuracy: 0.847656
Epoch 10, Batch 303 - Loss:   145.5731 Validation Accuracy: 0.847656
Epoch 10, Batch 304 - Loss:   166.7135 Validation Accuracy: 0.851562
Epoch 10, Batch 305 - Loss:   138.6474 Validation Accuracy: 0.847656
Epoch 10, Batch 306 - Loss:   240.4493 Validation Accuracy: 0.847656
Epoch 10, Batch 307 - Loss:   135.0242 Validation Accuracy: 0.847656
Epoch 10, Batch 308 - Loss:   349.9445 Validation Accuracy: 0.847656
Epoch 10, Batch 309 - Loss:   293.2075 Validation Accuracy: 0.847656
Epoch 10, Batch 310 - Loss:   320.1349 Validation Accuracy: 0.839844
Epoch 10, Batch 311 - Loss:   278.1601 Validation Accuracy: 0.839844
Epoch 10, Batch 312 - Loss:   226.8885 Validation Accuracy: 0.839844
Epoch 10, Batch 313 - Loss:   125.5888 Validation Accuracy: 0.832031
Epoch 10, Batch 314 - Loss:   164.3333 Validation Accuracy: 0.832031
Epoch 10, Batch 315 - Loss:   322.

Epoch 10, Batch 420 - Loss:   167.1504 Validation Accuracy: 0.843750
Epoch 10, Batch 421 - Loss:   393.8227 Validation Accuracy: 0.843750
Epoch 10, Batch 422 - Loss:   233.3181 Validation Accuracy: 0.843750
Epoch 10, Batch 423 - Loss:   266.0900 Validation Accuracy: 0.843750
Epoch 10, Batch 424 - Loss:   195.6492 Validation Accuracy: 0.843750
Epoch 10, Batch 425 - Loss:   186.8279 Validation Accuracy: 0.843750
Epoch 10, Batch 426 - Loss:   235.1970 Validation Accuracy: 0.847656
Epoch 10, Batch 427 - Loss:   243.5236 Validation Accuracy: 0.843750
Epoch 10, Batch 428 - Loss:   242.8977 Validation Accuracy: 0.847656
Epoch 10, Batch 429 - Loss:   210.2206 Validation Accuracy: 0.843750
Testing Accuracy: 0.875
